In [ ]:
!pip install beautifulsoup4

In [ ]:
!pip install lxml

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
def extract(page):
    # Google 'My User Agent', copy & paste it below
    headers = {'User-Agent': 'Paste your User Agent here'}
    
    url = f'https://malaysia.indeed.com/jobs?q=data+scientist&l=Malaysia&start={page}'  

    r = requests.get(url, headers)    
    soup = BeautifulSoup(r.content, 'html.parser')
    return soup

def transform(soup):    
    divs = soup.find_all('div', class_='job_seen_beacon')

    for item in divs:
        try:
            job_name = item.find('span')['title']
        except Exception as e:
            continue
        
        company = item.find('span', class_='companyName').text
        
        try:
            salary = item.find('div', class_='salary-snippet').span.text
        except:
            salary = 'Undisclosed'
        try:
            rating = item.find('span', class_='ratingNumber').text
        except:
            rating = 'None'
            
        
        summary = item.find('div', {'class' : 'job-snippet'}).text.strip().replace('\n','')
        post_date = item.find('span', class_='date').text.replace('Posted','')
               
        job = {
            'title' : job_name,
            'company' : company,
            'salary' : salary,
            'rating' : rating,
            'summary' : summary,  
            'post_date' : post_date,
        }
        
        joblist.append(job)    
            
    return

joblist = []    

# At the time of this project, only 24 pages of results available
for i in range(0, 220, 10):
    c = extract(0)
    transform(c)

df = pd.DataFrame(joblist)
df.to_csv('joblist.csv')